In [1]:
import torch
print(torch.cuda.is_available())           # True 表示有 GPU
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

True
NVIDIA GeForce RTX 5060 Laptop GPU


In [ ]:
#!/usr/bin/env python
# coding: utf-8

import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image
import io

MODEL_PATH = "/mnt/g/project/CXR_Analyzer/models/Lingshu-7B"
# MODEL_PATH = "G:\project\CXR_Analyzer\models\Lingshu-7B"

print("[VQA] loading...")
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
processor = AutoProcessor.from_pretrained(MODEL_PATH)
print("[VQA] loading completed")

In [ ]:
def vqa(image_bytes: bytes, question: str) -> str:
    """
    输入：
        image_bytes: 图像字节流（来自 Flask 上传）
        question: 用户提问（str）
    返回：
        answer: 模型回答（str）
    """
    try:
        # 字节流转 PIL.Image
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

        # 构造输入格式
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image},
                    {"type": "text", "text": question},
                ],
            }
        ]

        # 应用模板
        text = processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        image_inputs, video_inputs = process_vision_info(messages)

        # 构造模型输入
        inputs = processor(
            text=[text],
            images=image_inputs,
            videos=video_inputs,
            padding=True,
            return_tensors="pt",
        )
        inputs = inputs.to(model.device)

        # 生成回答
        generated_ids = model.generate(**inputs, max_new_tokens=50)
        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        output_text = processor.batch_decode(
            generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )
        return output_text[0].strip()

    except Exception as e:
        return f"[VQA error] {str(e)}"


In [ ]:
test_path = "/mnt/g/project/CXR_Analyzer/services/687754ce-7420bfd3-0a19911f-a27a3916-9019cd53.jpg"
with open(test_path, "rb") as f:
    image_bytes = f.read()

In [ ]:
question = "give me a report"
report = vqa(image_bytes,question)